In [ ]:
def read_transition_table(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            print(line.strip())
file_path = 'example1.txt'
read_transition_table(file_path)

q1,-,bRq2
q2,-,bRq1


#Turning Machine

In [ ]:
class TransitionRule:
    def __init__(self, current_state, input_symbol, input_symbol_value, direction, next_state):
        self.current_state = current_state
        self.input_symbol = input_symbol
        self.input_symbol_value = input_symbol_value
        self.direction = direction
        self.next_state = next_state

    def __str__(self):
        return f"('{self.current_state}', '{self.input_symbol}'): " \
               f"('{self.next_state}', '{self.input_symbol_value}', '{self.direction}')"

class TransitionTable:
    @staticmethod
    def parse_input_table(file_path, tape_symbols):
        transition_rules = []
        with open(file_path, 'r') as file:
            for line in file:
                parts = line.strip().split(',')
                current_state = parts[0]
                for i in range(1, len(tape_symbols) + 1):
                    if len(parts[i]) >= 4:
                        input_symbol_value = parts[i][0]
                        direction = parts[i][1]
                        next_state = parts[i][2:]
                        if input_symbol_value != '-':
                            rule = TransitionRule(current_state, tape_symbols[i-1], input_symbol_value, direction, next_state)
                            transition_rules.append(rule)
        return transition_rules

file_path = input("Enter the name of the .txt file: ")
while not file_path.endswith(".txt"):
    print("Please enter a file name ending with '.txt'.")
    file_path = input("Enter the name of the .txt file: ")

input_symbols = input("Enter input symbols separated by spaces: ").split()
tape_symbols = ['b'] + input_symbols

transition_rules = TransitionTable.parse_input_table(file_path, tape_symbols)

possible_states = sorted({rule.current_state for rule in transition_rules})

print("Possible states extracted from the table:", possible_states)

initial_state = input("Enter the initial state: ")
while initial_state not in possible_states:
    print("Invalid initial state. Please enter one of the possible states.")
    initial_state = input("Enter the initial state: ")

final_state = input("Enter the final state: ")
while final_state not in possible_states:
    print("Invalid final state. Please enter one of the possible states.")
    final_state = input("Enter the final state: ")

print("\nStates =", possible_states)
print("Input_symbols =", input_symbols)
print("Tape_symbols =", tape_symbols)
print("Transition_function = {")
for rule in transition_rules:
    print(f"('{rule.current_state}', '{rule.input_symbol}'): ('{rule.next_state}','{rule.input_symbol_value}', '{rule.direction}'),")
print("}")
print("Initial_state =", initial_state)
print("Blank_symbol =", 'b')
print("Final_states =", {final_state})

transition_function = {(rule.current_state, rule.input_symbol): (rule.next_state, rule.input_symbol_value, rule.direction) for rule in transition_rules}
blank_symbol = 'b'
final_states = {final_state}

Enter the name of the .txt file: example1.txt
Enter input symbols separated by spaces: 1
Possible states extracted from the table: ['q1', 'q2']
Enter the initial state: q1
Enter the final state: q1

States = ['q1', 'q2']
Input_symbols = ['1']
Tape_symbols = ['b', '1']
Transition_function = {
('q1', '1'): ('q2','b', 'R'),
('q2', '1'): ('q1','b', 'R'),
}
Initial_state = q1
Blank_symbol = b
Final_states = {'q1'}


In [ ]:
class TuringMachine:
    def __init__(self, states, input_symbols, blank_symbol, transition_function,
                 initial_state, final_states):
        self.states = states
        self.input_symbols = input_symbols
        self.tape_symbols = ['b'] + input_symbols
        self.transition_function = transition_function
        self.initial_state = initial_state
        self.current_state = initial_state
        self.blank_symbol = blank_symbol
        self.final_states = final_states

    def process_input(self, input_string):
        Tape = ['b'] * 8
        Head_pos = 0
        for i in range(len(input_string)):
            Tape[i] = input_string[i]
        print("Original Tape:", Tape, "with tapehead at index", Head_pos, "on state", self.current_state)
        while True:
            current_symbol = Tape[Head_pos]
            if self.current_state not in self.states:
                return False
            elif current_symbol not in self.tape_symbols:
                return False
            transition = self.transition_function.get((self.current_state, current_symbol))
            if transition:
                new_state, new_symbol, move = transition
                Tape[Head_pos] = new_symbol
                self.current_state = new_state
                if move == 'L':
                    Head_pos -= 1
                    if Head_pos == -1:
                        print("Tape head moved to index -1. Adding 'b' at the beginning and popping from the end.")
                        Tape.pop()
                        Tape.insert(0, 'b')
                        Head_pos = 0
                elif move == 'R':
                    Head_pos += 1
                    if Head_pos == len(Tape):
                        Tape.append('b')
                print(Tape, "with tapehead at index", Head_pos, "on state", self.current_state)
            else:
                break
        if self.current_state in self.final_states:
            return True
        return False

input_string = input("Enter the input string: ")
TM = TuringMachine(possible_states, input_symbols, blank_symbol, transition_function,
                   initial_state, final_states)
Result = TM.process_input(input_string)
if Result:
    print("Input string accepted by TM.")
else:
    print("Input string rejected by TM.")

Enter the input string: 1111
Original Tape: ['1', '1', '1', '1', 'b', 'b', 'b', 'b'] with tapehead at index 0 on state q1
['b', '1', '1', '1', 'b', 'b', 'b', 'b'] with tapehead at index 1 on state q2
['b', 'b', '1', '1', 'b', 'b', 'b', 'b'] with tapehead at index 2 on state q1
['b', 'b', 'b', '1', 'b', 'b', 'b', 'b'] with tapehead at index 3 on state q2
['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'] with tapehead at index 4 on state q1
Input string accepted by TM.


#UTM

In [ ]:
class Encoding:
    def __init__(self, turing_machine, input_string, delimiter="1"):
        self.turing_machine = turing_machine
        self.input_string = input_string
        self.delimiter = delimiter
        self.encoded_states = {}
        self.encoded_tape_symbols = {}
        self.encoded_direction = {'L': '0', 'R': '00'}
        for index, state in enumerate(self.turing_machine.states):
            self.encoded_states[state] = '0' * (index + 1)

        self.encoded_tape_symbols['b'] = '0'
        for index, tape_symbol in enumerate(self.turing_machine.tape_symbols):
            if tape_symbol != 'b':
                self.encoded_tape_symbols[tape_symbol] = '0' + '0' * index

    def encode_transition(self):
        encoded_description = str(self.delimiter * 3)

        for (current_state, current_symbol), (new_state, new_symbol, move) in self.turing_machine.transition_function.items():
            transition = ""
            transition += self.encoded_states.get(current_state, "")
            transition += self.delimiter
            transition += self.encoded_tape_symbols.get(current_symbol, "")
            transition += self.delimiter
            transition += self.encoded_states.get(new_state, "")
            transition += self.delimiter
            transition += self.encoded_tape_symbols.get(new_symbol, "")
            transition += self.delimiter
            transition += self.encoded_direction.get(move, "")
            transition += str(self.delimiter * 2)
            encoded_description += transition
        encoded_description += self.delimiter
        return encoded_description

    def encode_input_string(self):
        encoded_input = [self.encoded_tape_symbols.get(symbol, "") + self.delimiter for symbol in self.input_string[:-1]]
        encoded_input.append(self.encoded_tape_symbols.get(self.input_string[-1], ""))
        return "".join(encoded_input)

    def get_state_encoding(self):
        return self.encoded_states

    def get_tape_symbol_encoding(self):
        return self.encoded_tape_symbols

transition_rules, possible_states, input_symbols, tape_symbols, initial_state, final_state, input_string = TransitionTable.parse_input_table(file_path, tape_symbols), possible_states, input_symbols, tape_symbols, initial_state, final_state, input_string
transition_function = {(rule.current_state, rule.input_symbol): ( rule.next_state, rule.input_symbol_value, rule.direction) for rule in transition_rules}
blank_symbol = 'b'
final_states = {final_state}
TM = TuringMachine(possible_states, input_symbols, blank_symbol, transition_function,
                   initial_state, final_states)
encoder = Encoding(TM, input_string)
state_encoding = encoder.get_state_encoding()
print("State Encoding:", state_encoding)
encoded_transitions = encoder.encode_transition()
tape_symbol_encoding = encoder.get_tape_symbol_encoding()
print("Tape Symbol Encoding:", tape_symbol_encoding)
print("Encoded transitions:", encoded_transitions)
print("Input String: ", input_string)
encoded_input_string = encoder.encode_input_string() + encoder.delimiter
print("Encoded input string:", encoded_input_string)

State Encoding: {'q1': '0', 'q2': '00'}
Tape Symbol Encoding: {'b': '0', '1': '00'}
Encoded transitions: 11101001001010011001001010100111
Input String:  1111
Encoded input string: 001001001001


In [ ]:
import regex

class UniversalTuringMachine:
    def __init__(self, encoded_transitions, encoded_input_string, encoded_states, initial_state, final_state):
        self.encoded_transitions = encoded_transitions
        self.encoded_input_string = encoded_input_string
        self.tape1 = encoded_transitions
        self.tape2 = encoded_input_string
        self.tape2_head = 0
        self.tape3 = encoded_states[initial_state]
        self.current_state = initial_state
        self.final_state = encoded_states[final_state]
        self.delimiter = '1'

    def run(self):
        print("Initial Configuration:")
        print("Tape 1 (Encoded Transitions):", self.tape1)
        print("Tape 2 (Encoded Input String):", self.tape2)
        print("Tape 3 (Internal State):", self.tape3)

        while True:
            current_symbol = ""
            current_state = self.tape3
            symbol_index = self.tape2_head
            while symbol_index < len(self.tape2) and self.tape2[symbol_index] != self.delimiter:
                current_symbol += self.tape2[symbol_index]
                symbol_index += 1
            print("Current symbol: ", current_symbol)
            if current_symbol == "":
                break
            prefix = "11" + current_state + "1" + current_symbol
            transition = regex.findall(prefix + "10*10*10*", self.tape1)
            if not transition:
                print("No transition found for the current symbol and state. Halting and rejecting.")
                return False

            transition = transition[0]
            suffix = transition[len(prefix) + 1:]
            print("Transition Found for the current symbol: ")
            print("Prefix of current state and current symbol: ", prefix, "Suffix: ", suffix, "Transition: ", transition)
            new_state, new_symbol, direction = suffix.split(self.delimiter)
            print("New state:", new_state)
            print("New symbol:", new_symbol)
            print("Direction:", direction)
            self.tape2 = self.tape2[:symbol_index - len(current_symbol)] + new_symbol + self.tape2[symbol_index:]
            self.tape3 = new_state
            print("Updated Tape 2:", self.tape2)
            print("Current Configuration:")
            print("Tape 1 (Encoded Transitions):", self.tape1)
            print("Tape 2 (Encoded Input String):", self.tape2)
            print("Tape 3 (Internal State):", self.tape3)

            if direction == '0':
                self.tape2_head -= 1
                delimiter_count = 0
                while self.tape2_head >= 0 and delimiter_count < 2:
                    if self.tape2[self.tape2_head] == self.delimiter:
                        delimiter_count += 1
                    self.tape2_head -= 1
                self.tape2_head += 2
                return True
            elif direction == '00':
                self.tape2_head = symbol_index + 1
                if len(current_symbol) > len(new_symbol):
                    self.tape2_head -= len(current_symbol) - len(new_symbol)
                else:
                    self.tape2_head += len(new_symbol) - len(current_symbol)
                print("index: ", self.tape2_head)

        if self.tape3 == self.final_state:
            print("Reached final state. Halting and accepting.")
            return True
        if self.tape2 == self.delimiter:
            if self.tape3 == self.final_state:
                print("Reached end of input string. Final state reached. Halting and accepting.")
                return True
            else:
                print("Reached end of input string. Halting and rejecting.")
                return False

utm = UniversalTuringMachine(encoded_transitions, encoded_input_string, state_encoding, initial_state, final_state)
utm_result = utm.run()
if Result and utm_result:
    print("Input string accepted by both Turing machine and UTM.")
elif Result:
    print("Input string accepted by Turing machine but rejected by UTM.")
elif utm_result:
    print("Input string accepted by UTM but rejected by Turing machine.")
else:
    print("Input string rejected by both Turing machine and UTM.")

Initial Configuration:
Tape 1 (Encoded Transitions): 11101001001010011001001010100111
Tape 2 (Encoded Input String): 001001001001
Tape 3 (Internal State): 0
Current symbol:  00
Transition Found for the current symbol: 
Prefix of current state and current symbol:  110100 Suffix:  0010100 Transition:  11010010010100
New state: 00
New symbol: 0
Direction: 00
Updated Tape 2: 01001001001
Current Configuration:
Tape 1 (Encoded Transitions): 11101001001010011001001010100111
Tape 2 (Encoded Input String): 01001001001
Tape 3 (Internal State): 00
index:  2
Current symbol:  00
Transition Found for the current symbol: 
Prefix of current state and current symbol:  1100100 Suffix:  010100 Transition:  11001001010100
New state: 0
New symbol: 0
Direction: 00
Updated Tape 2: 0101001001
Current Configuration:
Tape 1 (Encoded Transitions): 11101001001010011001001010100111
Tape 2 (Encoded Input String): 0101001001
Tape 3 (Internal State): 0
index:  4
Current symbol:  00
Transition Found for the current sy